# Computation of cutting planes

# The set-up

In [1]:
import numpy as np
import accpm
%load_ext autoreload
%autoreload 1
%aimport accpm

$\DeclareMathOperator{\domain}{dom}
\newcommand{\transpose}{\text{T}}
\newcommand{\vec}[1]{\begin{pmatrix}#1\end{pmatrix}}$

# Introduction and setup

To test the computation of cutting planes we consider the problem
\begin{align*}
    &\text{minimize} \quad f_\text{obj}(x_0, x_1) = (x_0 - 5)^2 + (x_1 - 5)^2 \\
    &\phantom{\text{minimize}} \quad f_0(x_0, x_1) = 
    a_0^\transpose x - b_0 = \vec{1\\0}^\transpose \vec{x_0\\x_1} - 20 = x_0 - 20 \leq 0\\
    &\phantom{\text{minimize}} \quad f_1(x_0, x_1) =
    a_1^\transpose x - b_1 = \vec{-1\\0}^\transpose \vec{x_0\\x_1} = -x_0 \leq 0\\
    &\phantom{\text{minimize}} \quad f_2(x_0, x_1) =
    a_2^\transpose x - b_2 = \vec{0\\1}^\transpose \vec{x_0\\x_1} - 20 = x_1 - 20 \leq 0 \\
    &\phantom{\text{minimize}} \quad f_3(x_0, x_1) = 
    a_3^\transpose x - b_3 = \vec{0\\-1}^\transpose \vec{x_0\\x_1} = -x_1 \leq 0,
\end{align*}
which clearly has the solution $x^\star = (x_1^\star, x_2^\star) = (5, 5)$. Now, the gradients of the objective function and constraint functions are 
\begin{align*}
    &\nabla f_\text{obj}(x_0, x_1) = \vec{2(x_0 - 5)\\2(x_1 - 5)}, \\
    &\nabla f_0(x_0, x_1) = \vec{1\\0}, \quad \nabla f_1(x_0, x_1) = \vec{-1\\0}, \\
    &\nabla f_2(x_0, x_1) = \vec{0\\1}, \quad \nabla f_3(x_0, x_1) = \vec{0\\-1}.
\end{align*}
We implement these functions as follows:

In [2]:
def funcobj(x):
    return (x[0]-5)**2 + (x[1]-5)**2
    
def func0(x):
    return x[0] - 20

def func1(x):
    return -x[0]
    
def func2(x):
    return x[1] - 20
    
def func3(x):
    return -x[1]

def grad_funcobj(x):
    return np.array([2*(x[0] - 5), 2*(x[1] - 5)])

def grad_func0(x):
    return np.array([1, 0])

def grad_func1(x):
    return np.array([-1, 0])

def grad_func2(x):
    return np.array([0, 1])
    
def grad_func3(x):
    return np.array([0, -1])

A = np.array([[1, 0],[-1,0],[0,1],[0,-1]])
b = np.array([20, 0, 20, 0])

In [3]:
np.array([1,2]) is not None

True

# Analytic centers and cutting planes

The ACCPM requires that the initial polygon $\mathcal{P}_0$ (here I've abused terminology and by the initial polygon $\mathcal{P}_0$ I actually mean the system of linear inequalities $Ax \leq b$) contain the $\epsilon$-suboptimal set for some given $\epsilon > 0$. It is clear that if we take
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose}, b = \vec{20\\0\\20\\0}
\end{align*}
this will be satisfied for some reasonably small enough $\epsilon > 0$, as we know the solution is $x^\star = (x_0^\star, x_1^\star) = (5, 5) \in [0, 20] \times [0, 20]$. 

Now, we start with $k=0$.  
Now, $x^{(0)}_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=0}^{3}{\log{(b_i - a_i^\transpose x)}}.
\end{equation*}
So, we solve the problem
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{3
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_0}\begin{bmatrix}1\\0\end{bmatrix} + \frac{1}{x_0}\begin{bmatrix}-1\\0\end{bmatrix} + \frac{1}{20-x_1}\begin{bmatrix}0\\1\end{bmatrix} + \frac{1}{x_1}\begin{bmatrix}0\\-1\end{bmatrix} = 0 \\
    &\iff \frac{1}{20-x_0} - \frac{1}{x_0} = 0, \frac{1}{20-x_1} - \frac{1}{x_1} = 0 \\
    &\iff x_0 = \frac{20}{2} = 10, x_1 = \frac{20}{2} = 10,
\end{align*}
and conclude $x^{(0)}_{ac} = (10, 10)$. We then query the oracle at $x^{(0)}_{ac}$. (Here, $f_\text{best} = f_\text{obj}(10, 10) = 50$ since this is the $0$-th iteration.) We know the point is feasible and therefore
\begin{align*}
    &a_4 = \nabla f_\text{obj}(10, 10) = \vec{10\\10}, \\
    &b_4 = \nabla f_\text{obj}(10, 10)^\transpose \vec{10\\10} = \vec{10\\10}^\transpose \vec{10\\10} = 200,
\end{align*}
and so we update
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose\\a_4^\transpose}, b = \vec{20\\0\\20\\0\\200}, k = 1.
\end{align*}
Now, $x^{(1)}_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=0}^{4}{\log{(b_i - a_i^\transpose x)}}.
\end{equation*}
So, we solve the problem
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{4
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_0}\vec{1\\0} + \frac{1}{x_0}\vec{-1\\0} + \frac{1}{20-x_1}\vec{0\\1} + \frac{1}{x_1}\vec{0\\-1} + \frac{1}{200-10x_0-10x_1} \vec{10\\10} = 0\\
    &\iff \frac{1}{20-x_0} - \frac{1}{x_0} + \frac{1}{20 - x_0- x_1}= 0, \frac{1}{20-x_1} - \frac{1}{x_1} + \frac{1}{20 - x_0- x_1} = 0 \\
    &\iff x_0 = x_1 = 2(5 \pm \sqrt{5}) \approx 14.4721 \text{ or } 5.52786,
\end{align*}
and take $x^{(1)}_{ac} = (2(5-\sqrt{5}), 2(5-\sqrt{5})) \approx (5.52786, 5.52786)$. We then query the
oracle at $x^{(1)}_{ac}$. Here 
$f_\text{obj}(x^{(1)}_{ac}) = f_\text{obj}(2(5-\sqrt{5}), 2(5-\sqrt{5})) = 90 - 40\sqrt{5} \approx 0.557281 \leq f_\text{best} = 50$ so we update 
$f_\text{best} = 90 - 40\sqrt{5} \approx 0.557281$ and therefore put  
\begin{align*}
    &a_5 = \nabla f_\text{obj}(x^{(1)}_{ac}) = 
           \nabla f_\text{obj}\vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} =
           \vec{10-4\sqrt{5}\\10-4\sqrt{5}} \approx 
           \vec{1.05573\\1.05573}, \\
    &b_5 = \nabla f_\text{obj}(x^{(1)}_{ac})^\transpose \vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} = 
           \vec{10-4\sqrt{5}\\10-4\sqrt{5}}^\transpose \vec{2(5-\sqrt{5}) \\ 2(5-\sqrt{5})} = 4 (10 - 4 \sqrt{5}) (5 - \sqrt{5}) = 40 (7 - 3\sqrt{5}) \approx 11.6718,
\end{align*}
updating $A$ and $b$ also. Iteration $k = 1$ is concluded by incrementing $k$.

For $k = 2$ we similarly solve 
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{5
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_0}\vec{1\\0} + \frac{1}{x_0}\vec{-1\\0} + \frac{1}{20-x_1}\vec{0\\1} + \frac{1}{x_1}\vec{0\\-1} + \frac{1}{200-10x_0-10x_1} \vec{10\\10}
    + \frac{1}{40(7-3\sqrt{5})-(10-4\sqrt{5})x_0 - (10-4\sqrt{5})x_1}  \vec{10-4\sqrt{5}\\10-4\sqrt{5}} = 0\\
    &\iff \frac{1}{20-x_0} - \frac{1}{x_0} + \frac{1}{20 - x_0- x_1} + \frac{10-4\sqrt{5}}{40(7-3\sqrt{5})-(10-4\sqrt{5})x_0 - (10-4\sqrt{5})x_1} = 0
    , \frac{1}{20-x_1} - \frac{1}{x_1} + \frac{1}{20 - x_0- x_1} + \frac{10-4\sqrt{5}}{40(7-3\sqrt{5})-(10-4\sqrt{5})x_0 - (10-4\sqrt{5})x_1} = 0 \\
    &\iff x_0 = x_1 = 2(5 \pm \sqrt{5}) \approx 14.4721 \text{ or } 5.52786,
\end{align*}

In [55]:
 accpm.accpm(funcobj, (func0, func1, func2, func3), A, b, 
           grad_func=grad_funcobj, 
           grad_constr=(grad_func0, grad_func1, grad_func2, grad_func3),
           alpha=0.01, beta=0.7, start=1, maxiter = 15)

    SUCCESS with i = 5
******** At iteration 0 AC computation SUCCESSFUL!
******** At iteration 0 AC is [ 9.99999996  9.99999996]
    SUCCESS with i = 7
******** At iteration 1 AC computation SUCCESSFUL!
******** At iteration 1 AC is [ 5.52787706  5.52783738]
    SUCCESS with i = 6
******** At iteration 2 AC computation SUCCESSFUL!
******** At iteration 2 AC is [ 3.02594683  3.02624606]
    SUCCESS with i = 8
******** At iteration 3 AC computation SUCCESSFUL!
******** At iteration 3 AC is [ 4.76614104  4.76525504]
    SUCCESS with i = 10
******** At iteration 4 AC computation SUCCESSFUL!
******** At iteration 4 AC is [ 5.15324012  5.25818967]
    SUCCESS with i = 6
******** At iteration 5 AC computation SUCCESSFUL!
******** At iteration 5 AC is [ 7.88341122  2.3826449 ]
    SUCCESS with i = 8
******** At iteration 6 AC computation SUCCESSFUL!
******** At iteration 6 AC is [ 5.22551073  4.76114569]
    SUCCESS with i = 7
******** At iteration 7 AC computation SUCCESSFUL!
******** At ite